In [ ]:
import matplotlib.pyplot as plt

import numpy as np

from tensorflow.keras.utils import load_img, img_to_array

from keras.utils import image_dataset_from_directory
from keras import callbacks, Input, layers, Model
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
train_dataset = image_dataset_from_directory('./train',
                                             image_size=(200,200),batch_size=2)
validation_dataset = image_dataset_from_directory('./val',
                                             image_size=(200,200),batch_size=2)

In [ ]:
inputs = Input(shape=(200,200,3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=8,kernel_size=3,activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=16,kernel_size=3,activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=32,kernel_size=3,activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64,kernel_size=3,activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128,kernel_size=3,activation='relu')(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1,activation='sigmoid')(x)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')

In [ ]:
early_stop = EarlyStopping(monitor='vall_accuracy', patience=7, 
                           mode='max', verbose=1)

checkpoint = ModelCheckpoint('./regressor-{epoch:02d}-{val_accuracy:.2f}.hdf5', 
                             monitor='val_accuracy', verbose=1,
                             mode='max', save_best_only=True)

callbacks_list = [early_stop, checkpoint]

In [ ]:
history = model.fit(train_dataset,epochs=30,callbacks=callbacks_list,validation_data=validation_dataset)

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

epochs=range(1,len(accuracy)+1)

In [ ]:
plt.title('Accuracy (Train & Val Sets)')
plt.plot(epochs,accuracy,color='green',label='Traininig Accuracy')
plt.plot(epochs,val_accuracy,color='red',label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
model.load_weights('./regressor-01-0.50.hdf5')

In [ ]:
test_dataset = image_dataset_from_directory('./test',
                                             image_size=(200,200),batch_size=2)
model.evaluate(test_dataset)

In [ ]:
test_image = load_img('./test/chair/1.jpg', target_size=(200, 200))
plt.axis('off')
plt.imshow(test_image)

In [ ]:
x = img_to_array(test_image)
x.shape

In [ ]:
x = np.expand_dims(x, axis=0)

In [ ]:
prediction = model.predict(x)

In [ ]:
prediction[0][0]